<a href="https://colab.research.google.com/github/vincent4u/Assignment-Code/blob/master/look_into_this.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, KFold

# Load the text data
X = ["This is a hate speech tweet.", "I love this product.", "Minorities are not welcome here.", "The service was great!"]
y = [1, 0, 1, 0]  # 1 for hate speech, 0 for non-hate speech

# Extract text features using TF-IDF
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

# Perform cross-validation to evaluate the GMM model
kf = KFold(n_splits=1, shuffle=True, random_state=42)
accuracies = []
precisions = []
recalls = []
f1_scores = []

for train_index, test_index in kf.split(X_vectorized):
    X_train, X_test = X_vectorized[train_index], X_vectorized[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]

    # Train the Gaussian Mixture Model
    gmm = GaussianMixture(n_components=2, covariance_type='full')
    gmm.fit(X_train.toarray())

    # Make predictions on the test data
    y_pred = gmm.predict(X_test.toarray())

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Print the average performance metrics
print(f"Accuracy: {np.mean(accuracies):.2f}")
print(f"Precision: {np.mean(precisions):.2f}")
print(f"Recall: {np.mean(recalls):.2f}")
print(f"F1-score: {np.mean(f1_scores):.2f}")

In [15]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [18]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import numpy as np
import os
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix



import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor

from scipy.stats import pearsonr
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def preprocess_text(text):
    # Remove specific tokens
    text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")


    # Remove special characters and punctuation marks using regular expressions
    text = re.sub(r'[^\w\s]', '', text)

    # Remove URLs, email addresses, and phone numbers
    text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)

    # Remove numbers
    text = re.sub(r"\d+", "", text)

    # Remove non-ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # Convert the text to lowercase
    text = text.lower()

    # Remove extra white spaces
    text = re.sub(r"\s+", " ", text)

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a single string
    preprocessed_text = " ".join(tokens)

    return preprocessed_text

# Preprocess the text
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

print(train_df['preprocessed_comment'])

0       kindergarden ignorantsve added pakistani artic...
1                   violate rrv rule may banned wikipedia
2                     judging message tp could use advice
3                                     love fatty sex mmmm
4        please could explain thinking warning page thank
                              ...                        
8694    utci beg differ history hugely important licen...
8695           penis sex grawp haeppenispenisvaginavagina
8696                      thats exactly gamer dying breed
8697                          article unless fact support
8698                milton friedman lp also measure thing
Name: preprocessed_comment, Length: 8699, dtype: object


In [22]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora
from gensim.models import LdaMulticore
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

X_train = train_df['preprocessed_comment'].tolist()
y_train = train_df['toxicity'].tolist()

X_val = val_df['preprocessed_comment'] .tolist()
y_val = val_df['toxicity'].tolist()

# Tokenize the text
tokenizer = lambda text: text.split()
X_train_tokens = [tokenizer(text) for text in X_train]
X_val_tokens = [tokenizer(text) for text in X_val]

# Create the bag-of-words representation
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_val_bow = vectorizer.transform(X_val)

# Create the LDA model
dictionary = corpora.Dictionary(X_train_tokens)
corpus = [dictionary.doc2bow(text) for text in X_train_tokens]
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=2, passes=10, iterations=500)

# Get the topic distributions for the training and validation data
X_train_topics = [lda_model[doc] for doc in corpus]
X_val_topics = [lda_model[dictionary.doc2bow(text)] for text in X_val_tokens]

# Pad the topic distributions to a fixed length
max_topics = max(len(doc_topics) for doc_topics in X_train_topics)
X_train_topics_padded = [doc_topics + [0] * (max_topics - len(doc_topics)) for doc_topics in X_train_topics]
X_val_topics_padded = [doc_topics + [0] * (max_topics - len(doc_topics)) for doc_topics in X_val_topics]

# Convert to numpy arrays
X_train_topics_padded = np.array(X_train_topics_padded)
X_val_topics_padded = np.array(X_val_topics_padded)

# Train a classifier on the padded topic distributions
clf = LogisticRegression()
clf.fit(X_train_topics_padded, y_train)

# Evaluate the model on the validation data
y_pred = clf.predict(X_val_topics_padded)

accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (8699, 2) + inhomogeneous part.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

X_train = train_df['preprocessed_comment'].tolist()
y_train = train_df['toxicity'].tolist()

X_val = val_df['preprocessed_comment'] .tolist()
y_val = val_df['toxicity'].tolist()

# Tokenize the text
tokenizer = lambda text: text.split()
X_train_tokens = [tokenizer(text) for text in X_train]
X_val_tokens = [tokenizer(text) for text in X_val]

# Create the bag-of-words representation
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_val_bow = vectorizer.transform(X_val)

# Train the Naive Bayes model
clf = MultinomialNB()
clf.fit(X_train_bow, y_train)

# Evaluate the model on the validation data
y_pred = clf.predict(X_val_bow)

accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

In [26]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text (assuming you have a preprocess_text function)
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

X_train = train_df['preprocessed_comment'].tolist()
y_train = train_df['toxicity'].tolist()

X_val = val_df['preprocessed_comment'].tolist()
y_val = val_df['toxicity'].tolist()

# Create the bag-of-words representation
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_val_bow = vectorizer.transform(X_val)

# Train the Naive Bayes model
clf = MultinomialNB()
clf.fit(X_train_bow, y_train)

# Evaluate the model on the validation data
y_pred = clf.predict(X_val_bow)

accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

Accuracy: 0.87
Precision: 0.21
Recall: 0.01
F1-score: 0.02


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text (assuming you have a preprocess_text function)
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

X_train = train_df['preprocessed_comment'].tolist()
y_train = train_df['toxicity'].tolist()

X_val = val_df['preprocessed_comment'].tolist()
y_val = val_df['toxicity'].tolist()

# Create the bag-of-words representation
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_val_bow = vectorizer.transform(X_val)

# Define the parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]
}

# Create the Naive Bayes classifier
clf = MultinomialNB()

# Perform GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train_bow, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the validation data
y_pred = best_model.predict(X_val_bow)

accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

In [31]:
!pip install transformers

In [58]:
#hybrid model


import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.base import TransformerMixin

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Implement your text preprocessing function here
    text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters and punctuation
    text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)  # Remove URLs, email addresses, and phone numbers
    text = re.sub(r"\d+", "", text)  # Remove numbers
    text = text.encode("ascii", "ignore").decode()  # Remove non-ASCII characters
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra white spaces

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stop words and apply stemming
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

# Feature extraction
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_df['preprocessed_comment'])
X_val = vectorizer.transform(val_df['preprocessed_comment'])
y_train = train_df['toxicity']
y_val = val_df['toxicity']

# Fit the LDA model on the entire training data
lda = LatentDirichletAllocation(n_components=10, random_state=42)
X_train_lda = lda.fit_transform(X_train)
X_val_lda = lda.transform(X_val)

# Naive Bayes Classifier
nb = MultinomialNB()
nb.fit(X_train, y_train)

# XGBoost Classifier
xgb = XGBClassifier(objective='binary:logistic', random_state=42)
xgb.fit(X_train_lda, y_train)

# Create a custom transformer to apply the Naive Bayes model
class NBTransformer(TransformerMixin):
    def __init__(self, nb_model):
        self.nb_model = nb_model

    def fit(self, X, y=None):
        self.nb_model.fit(X, y)
        return self

    def transform(self, X):
        return self.nb_model.predict_proba(X)

# Create the hybrid model
hybrid_model = Pipeline([
    ('features', FeatureUnion([
        ('lda', lda),
        ('nb', NBTransformer(nb))
    ])),
    ('xgb', xgb)
])

# Fit the hybrid model
hybrid_model.fit(X_train, y_train)

# Evaluate the hybrid model
y_pred = hybrid_model.predict(X_val)

# Calculate performance metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")


Accuracy: 0.85
Precision: 0.19
Recall: 0.04
F1-score: 0.07


In [ ]:
#LDA model alone

import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from tqdm.auto import tqdm
from joblib import Parallel, delayed

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

# Create the CountVectorizer and LDA pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('lda', LatentDirichletAllocation(n_components=10, random_state=42))
])

# Fit and transform the data in parallel
X_train = pipeline.fit_transform(train_df['preprocessed_comment'])
X_val = pipeline.transform(val_df['preprocessed_comment'])

y_train = train_df['toxicity']
y_val = val_df['toxicity']

# Evaluate the model in parallel
def evaluate_model(y_val, y_pred_labels):
    accuracy = accuracy_score(y_val, y_pred_labels)
    precision = precision_score(y_val, y_pred_labels, average='macro', zero_division='warn')
    recall = recall_score(y_val, y_pred_labels, average='macro', zero_division='warn')
    f1 = f1_score(y_val, y_pred_labels, average='macro')
    return accuracy, precision, recall, f1

y_pred = pipeline.transform(val_df['preprocessed_comment'])
y_pred_labels = np.argmax(y_pred, axis=1)

accuracy, precision, recall, f1 = evaluate_model(y_val, y_pred_labels)

print(f"accuracy = {accuracy:.2f}")
print(f"precision = {precision:.2f}")
print(f"recall = {recall:.2f}")
print(f"f1 = {f1:.2f}")

In [ ]:
#Naive Bayes  only

import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from tqdm.auto import tqdm
from joblib import Parallel, delayed

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

# Create the Naive Bayes pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])

# Fit the pipeline
pipeline.fit(train_df['preprocessed_comment'], train_df['toxicity'])

# Predict and evaluate
y_pred = pipeline.predict(val_df['preprocessed_comment'])

accuracy = accuracy_score(val_df['toxicity'], y_pred)
precision = precision_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
recall = recall_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
f1 = f1_score(val_df['toxicity'], y_pred, average='macro')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


In [57]:
#LDA and Logistic regression

import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.auto import tqdm

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

# Create the LDA pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('lda', LatentDirichletAllocation(n_components=10, random_state=42))  # Adjust n_components as needed
])

# Fit and transform the data
X_train = pipeline.fit_transform(train_df['preprocessed_comment'])
X_val = pipeline.transform(val_df['preprocessed_comment'])

# Your training and validation labels
y_train = train_df['toxicity']
y_val = val_df['toxicity']

# Train Logistic Regression classifier
classifier = LogisticRegression(random_state=42)
classifier.fit(X_train, y_train)

# Predict on validation set
y_pred = classifier.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='macro', zero_division='warn')
recall = recall_score(y_val, y_pred, average='macro', zero_division='warn')
f1 = f1_score(y_val, y_pred, average='macro')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

Accuracy: 0.87
Precision: 0.43
Recall: 0.50
F1 Score: 0.46


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
